In [1]:

import  numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import xgboost as xgb 
from sklearn import metrics as m
from thundersvm import SVC as svmgpu
import calculateWeightUsingGa2 as aresult
import pandas as pd
import itertools
import random
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder

def swapcolumns(trainval, testval, coldindexval):
    trainval[trainval != coldindexval] = 5
    testval[testval != coldindexval] = 5

    trainval[trainval == coldindexval] = 0
    trainval[trainval == 5] = 1

    testval[testval == coldindexval] = 0
    testval[testval == 5] = 1

    return trainval, testval


randomseed=42
np.random.seed(randomseed)

In [2]:

xtest=np.array(pd.read_csv('../dataset/xtest.txt'))
xtrain=np.array(pd.read_csv('../dataset/xtrain.txt'))
ytest_original=np.array(pd.read_csv('../dataset/ytest.txt')).ravel()
ytrain_original =np.array(pd.read_csv('../dataset/ytrain.txt')).ravel()

# data=datasets.load_wine()
# x=data.data
# y=data.target

# data = pd.read_csv("../dataset/seeds_dataset.txt", sep="\t", header=None)
# data = shuffle(data)
# le = LabelEncoder()
# data.iloc[:, -1] = le.fit_transform(data.iloc[:, -1])
# x = data.iloc[:, :-1]
# y = data.iloc[:, -1]
# print(np.unique(y))

# data = pd.read_csv("../dataset/ionosphere.data",  header=None)
# data = shuffle(data)
# le = LabelEncoder()
# data.iloc[:, -1] = le.fit_transform(data.iloc[:, -1])
# x = data.iloc[:, :-1]
# y = data.iloc[:, -1]
# print(np.unique(y))




# xtrain,xtest,ytrain_original,ytest_original=train_test_split(x,y,random_state=randomseed,test_size=0.3) 

ytrain=ytrain_original.copy()
ytest=ytest_original.copy() 

# ytrain, ytest = swapcolumns(ytrain, ytest, 2)

In [3]:


clf=[]
acc=[]
finalacc=[]
ypredproba_all=[]
ypredconfprob_all=[]


#================================================= 

# Class 1
# ===========================
ytrain = ytrain_original.copy()
ytest = ytest_original.copy()
ytrain,ytest= swapcolumns(ytrain,ytest,1)
#=================================================

rf=RandomForestClassifier(random_state=randomseed, n_estimators=10)
rf.fit(xtrain,ytrain)
print('original score',m.f1_score(ytest,rf.predict(xtest),average='weighted'))


xgbc=xgb.XGBClassifier(random_state=randomseed,n_estimators=100)
xgbc.fit(xtrain,ytrain)
xgbpred=xgbc.predict(xtest)
print('original score 2',m.f1_score(ytest,xgbpred,average='weighted'))


#================================================= 

rf=RandomForestClassifier(random_state=randomseed, n_estimators=10)
rf.fit(xtrain,ytrain)
rfpred=rf.predict(xtest)
print(m.f1_score(ytest,rfpred,average='weighted'))

clf.append(rf)
acc.append(m.f1_score(ytest,rfpred,average='weighted'))
ypredproba_all.append(rf.predict_proba(xtest)[:,0])

confmat = m.confusion_matrix(ytest, rfpred)
confsumh = np.sum(confmat, axis=0)
propconfmat = confmat.copy()
for i in range(propconfmat.shape[0]):
    propconfmat[:, i] = 100 * propconfmat[:, i] / confsumh[i]
ypredconfprob_all.append(propconfmat.ravel() / 100)


#=================================================
svc=svmgpu(random_state=randomseed,probability=True,C=100,gamma=0.0001)
svc.fit(xtrain,ytrain)

svcpred=svc.predict(xtest)
print(m.f1_score(ytest,svcpred,average='weighted'))

clf.append(svc)
acc.append(m.f1_score(ytest,svcpred,average='weighted'))
ypredproba_all.append(svc.predict_proba(xtest)[:,0])

confmat = m.confusion_matrix(ytest, rfpred)
confsumh = np.sum(confmat, axis=0)
propconfmat = confmat.copy()
for i in range(propconfmat.shape[0]):
    propconfmat[:, i] = 100 * propconfmat[:, i] / confsumh[i]
ypredconfprob_all.append(propconfmat.ravel() / 100)

#=================================================
xgbc=xgb.XGBClassifier(random_state=randomseed,n_estimators=100)
xgbc.fit(xtrain,ytrain)

xgbpred=xgbc.predict(xtest)
print(m.f1_score(ytest,xgbpred,average='weighted'))


clf.append(xgbc)
acc.append(m.f1_score(ytest,xgbpred,average='weighted'))
ypredproba_all.append(xgbc.predict_proba(xtest)[:,0])

confmat = m.confusion_matrix(ytest, rfpred)
confsumh = np.sum(confmat, axis=0)
propconfmat = confmat.copy()
for i in range(propconfmat.shape[0]):
    propconfmat[:, i] = 100 * propconfmat[:, i] / confsumh[i]
ypredconfprob_all.append(propconfmat.ravel() / 100)

#=================================================
#=================================================
# generate combinations of features 12,6
comb = list(itertools.combinations(np.arange(0, 12, 1), 7))

# generate 50 random numbers
randnums = []
for i in range(50):
    randnums.append(random.randrange(0, len(comb)))

print(randnums)

comb = np.array(comb)[randnums, :]


for i in range(len(comb)):
    print(i, " ==================== ", comb[i])

    rf = RandomForestClassifier(random_state=randomseed, n_estimators=50)
    rf.fit(xtrain[:, comb[i]], ytrain)
    rfpred = rf.predict(xtest[:, comb[i]])
    print(m.f1_score(ytest, rfpred,average='weighted'))

    clf.append(rf)
    acc.append(m.f1_score(ytest, rfpred,average='weighted'))
    ypredproba_all.append(rf.predict_proba(xtest[:, comb[i]])[:,0])

    confmat = m.confusion_matrix(ytest, rfpred)
    confsumh = np.sum(confmat, axis=0)
    propconfmat = confmat.copy()
    for i in range(propconfmat.shape[0]):
        propconfmat[:, i] = 100 * propconfmat[:, i] / confsumh[i]
    ypredconfprob_all.append(propconfmat.ravel() / 100)

    xgbmodel = xgb.XGBClassifier(random_state=randomseed, n_estimators=50)
    xgbmodel.fit(xtrain, ytrain)
    xgbmodelpred = xgbmodel.predict(xtest)
    print(m.f1_score(ytest, xgbmodelpred,average='weighted'))

    clf.append(xgbmodel)
    acc.append(m.f1_score(ytest, xgbmodelpred,average='weighted'))
    ypredproba_all.append(xgbmodel.predict_proba(xtest)[:,0])

    confmat = m.confusion_matrix(ytest, xgbmodelpred)
    confsumh = np.sum(confmat, axis=0)
    propconfmat = confmat.copy()
    for i in range(propconfmat.shape[0]):
        propconfmat[:, i] = 100 * propconfmat[:, i] / confsumh[i]
    ypredconfprob_all.append(propconfmat.ravel() / 100)

# #=================================================


original score 0.6765880295918836
original score 2 0.7315606204495092
0.6765880295918836
0.5333333333333333
0.7315606204495092
[221, 646, 601, 182, 85, 781, 113, 380, 373, 204, 513, 660, 311, 710, 648, 165, 729, 658, 76, 503, 179, 752, 27, 51, 392, 464, 757, 192, 6, 263, 453, 500, 671, 627, 616, 463, 176, 582, 786, 659, 482, 357, 186, 405, 672, 98, 183, 101, 506, 335]
0  ====================  [ 0  1  4  6  7 10 11]
0.6848088520187307
0.738221824835998
1  ====================  [ 1  4  5  6  7  8 11]
0.664513297603802
0.738221824835998
2  ====================  [ 1  3  4  5  7  9 10]
0.6665071770334928
0.738221824835998
3  ====================  [ 0  1  3  6  7  8 10]
0.7280303850673597
0.738221824835998
4  ====================  [ 0  1  2  4  7  9 11]
0.7145645605497474
0.738221824835998
5  ====================  [ 3  5  6  8  9 10 11]
0.5725761396113839
0.738221824835998
6  ====================  [ 0  1  2  6  7  8 11]
0.7147332386315894
0.738221824835998
7  ====================  [ 0  3  4 

In [4]:
# pd_clf=pd.DataFrame(list(clf))
pd_acc=pd.DataFrame(acc) 
pd_ypredproba_all=pd.DataFrame(np.array(ypredproba_all))
pd_ypredconfprob_all=pd.DataFrame(ypredconfprob_all)

In [5]:
# classname='class1.txt'
# pd_acc.to_csv('pd_acc_'+classname,sep=',',index=False) 
# pd_ypredproba_all.to_csv('pd_ypredproba_all_'+classname,sep=',',index=False)
# pd_ypredconfprob_all.to_csv('pd_ypredconfprob_all_'+classname,sep=',',index=False)

In [6]:
# pd.DataFrame(ytrain_original).to_csv('ytrain_original.txt',sep=',',index=False)
# pd.DataFrame(ytest).to_csv('ytest_class1.txt',sep=',',index=False)

In [7]:
pd_ypredconfprob_all

,0,1,2,3
0,0.50,0.20,0.49,0.79
1,0.50,0.20,0.49,0.79
2,0.50,0.20,0.49,0.79
3,0.51,0.21,0.48,0.78
4,0.62,0.20,0.37,0.79
...,...,...,...,...
98,0.62,0.20,0.37,0.79
99,0.48,0.23,0.51,0.76
100,0.62,0.20,0.37,0.79
101,0.51,0.21,0.48,0.78


In [ ]:
ytest

In [9]:

import calculateWeightUsingGa2 as aresult
weightvalga=aresult.getbestvalues(acc)

temp=pd.DataFrame(ypredproba_all)


0
1
2
3
4
5
6
7
8
9
43.93013316097456 [0.41140438 0.01280603 0.67483366 0.48084834 0.68132484 0.3041915
 0.68437426 0.36780403 0.68644215 0.66007689 0.70394984 0.59795273
 0.70537776 0.12726387 0.70748809 0.5991597  0.71081271 0.48436685
 0.72260621 0.441991   0.72703178 0.59179905 0.73623872 0.23044814
 0.74260356 0.30724712 0.76318204 0.49651784 0.76382703 0.12932833
 0.76782584 0.17102952 0.76932724 0.53123324 0.77018385 0.13909364
 0.7812095  0.25079033 0.7827234  0.58202056 0.78928821 0.16567919
 0.80023815 0.52911497 0.80434113 0.02319344 0.81248741 0.67897888
 0.81324865 0.56987058 0.81557063 0.49568231 0.82558293 0.15439133
 0.83814277 0.06593889 0.84075443 0.5486409  0.84447656 0.62960242
 0.84597175 0.56534413 0.84905529 0.49595074 0.86556517 0.36081984
 0.87212412 0.15148418 0.87227594 0.20815398 0.87804005 0.24691778
 0.88432151 0.39337011 0.9066409  0.56679171 0.91183152 0.44595808
 0.91720863 0.00819398 0.9212621  0.15840174 0.95086907 0.22200106
 0.95319609 0.41228595 0.

In [15]:
print(np.array(ypredproba_all).shape)
print(len(acc))

(103, 600)
103


In [18]:
finalval=0
for i in range(len(acc)):
    
    finalval += weightvalga[i]* np.column_stack( (temp.iloc[i,:],1- temp.iloc[i,:]))
        
    print(
        m.f1_score(ytest, np.argmax(np.column_stack( (temp.iloc[i,:],1- temp.iloc[i,:])), axis=1),average='weighted'),
        acc[i],
    )
    

print('f1_score',m.f1_score(ytest,np.argmax(finalval,axis=1),average='weighted'))
print('accuracy_score',m.accuracy_score(ytest,np.argmax(finalval,axis=1)))
    


0.6765880295918836 0.6765880295918836
0.16666666666666666 0.5333333333333333
0.7315606204495092 0.7315606204495092
0.6848088520187307 0.6848088520187307
0.738221824835998 0.738221824835998
0.664513297603802 0.664513297603802
0.738221824835998 0.738221824835998
0.6665071770334928 0.6665071770334928
0.738221824835998 0.738221824835998
0.7280303850673597 0.7280303850673597
0.738221824835998 0.738221824835998
0.7145645605497474 0.7145645605497474
0.738221824835998 0.738221824835998
0.5725761396113839 0.5725761396113839
0.738221824835998 0.738221824835998
0.7147332386315894 0.7147332386315894
0.738221824835998 0.738221824835998
0.6848088520187307 0.6848088520187307
0.738221824835998 0.738221824835998
0.6802147391292281 0.6802147391292281
0.738221824835998 0.738221824835998
0.7129436588130345 0.7129436588130345
0.738221824835998 0.738221824835998
0.6556712962962963 0.6556712962962963
0.738221824835998 0.738221824835998
0.6657722804442252 0.6657722804442252
0.738221824835998 0.738221824835998

In [12]:
# finalval=[0,]
# for i in range(len(acc)):
#     finalval += [weightvalga[i]*ypredproba_all[i],weightvalga[i]* (1-ypredproba_all[i])]

# print('f1_score',m.f1_score(ytest,np.argmax(finalval,axis=1),average='weighted'))
# print('accuracy_score',m.accuracy_score(ytest,np.argmax(finalval,axis=1)))
    

In [13]:
np.array([ypredproba_all[0],(1-ypredproba_all[0])]).reshape(600,2)

array([[0.2, 0.4],
       [0.7, 0.4],
       [0.2, 0.4],
       ...,
       [0.8, 0.8],
       [0.7, 0.9],
       [0.3, 0.7]])

In [14]:
np.column_stack( (ypredproba_all[0,:],1- ypredproba_all[0,:]))

TypeError: list indices must be integers or slices, not tuple

In [ ]:
temp=pd.DataFrame(ypredproba_all)

In [ ]:
np.column_stack( (temp.iloc[0,:],1- temp.iloc[0,:])) + 

In [ ]:
finalval=0
for i in range(len(acc)):
    
    finalval += ypredproba_all[i]
        
    print(
        m.accuracy_score(ytest, np.argmax(ypredproba_all[i], axis=1)),
        acc[i],
    )
    

print('f1_score',m.f1_score(ytest,np.argmax(finalval,axis=1),average='weighted'))
print('accuracy_score',m.accuracy_score(ytest,np.argmax(finalval,axis=1)))
    


In [ ]:
np.argmax(ypredproba_all[0],axis=1)

In [19]:
svc=svmgpu(random_state=randomseed,probability=True,C=100,gamma=0.0001)
svc.fit(xtrain,ytrain)

svcpred=svc.predict(xtest)
print(m.f1_score(ytest,svcpred,average='weighted'))

temppp=svc.predict_proba(xtest)

# clf.append(svc)
# acc.append(m.f1_score(ytest,svcpred,average='weighted'))
# ypredproba_all.append(svc.predict_proba(xtest))

# confmat = m.confusion_matrix(ytest, rfpred)
# confsumh = np.sum(confmat, axis=0)
# propconfmat = confmat.copy()
# for i in range(propconfmat.shape[0]):
#     propconfmat[:, i] = 100 * propconfmat[:, i] / confsumh[i]
# ypredconfprob_all.append(propconfmat / 100)


0.5333333333333333


In [20]:
np.argmax(temppp,axis=1)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [21]:
temppp

array([[0.72328657, 0.27671343],
       [0.65977824, 0.3402218 ],
       [0.64820737, 0.35179263],
       ...,
       [0.68141174, 0.31858823],
       [0.69823307, 0.3017669 ],
       [0.6632911 , 0.3367089 ]], dtype=float32)

In [25]:
m.f1_score(ytest, np.argmax(1-temppp, axis=1),average='weighted')

0.5333333333333333